In [1]:
!pip install tpot

     ---------------------------------------- 87.2/87.2 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 78.5/78.5 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 10.3/10.3 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 89.1/89.1 MB 959.1 kB/s eta 0:00:00
     -------------------------------------- 499.4/499.4 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 96.0/96.0 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 140.6/140.6 kB 2.1 MB/s eta 0:00:0

  DEPRECATION: stopit is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: deap is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import re
#visual
import matplotlib.pyplot as plt

#modeling
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

from skimage.feature import greycomatrix,greycoprops
from skimage import io

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from tpot import TPOTClassifier

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
df_1 = pd.read_csv('full_train_dt.csv')

df_1

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,Energy,Corr,Diss_sim,Homogen,Contrast,Energy2,...,47699dense,47720dense,47932dense,48142dense,48143dense,48241dense,49149dense,49260dense,49634dense,49784dense
0,0,0,0,0,0.569125,0.995196,1.267977,0.787601,29.518778,0.557591,...,0.029683,1.985881,0.000000,0.041704,0.000000,0.000000,0.455218,0.000057,0.011483,0.266430
1,1,1,1,1,0.527938,0.995842,1.471252,0.822016,57.029869,0.515226,...,0.030147,1.024253,0.055374,0.051034,0.000000,0.000000,0.645608,0.000091,0.010660,0.170045
2,2,2,2,2,0.545918,0.996925,1.381366,0.830510,46.555213,0.534082,...,0.029008,0.367422,0.500729,0.030002,0.000000,0.003778,0.459859,0.000161,0.009236,0.206810
3,3,3,3,3,0.596634,0.996485,1.356422,0.838555,50.394299,0.585423,...,0.030465,0.079529,0.000000,0.061716,0.000000,0.000195,0.481287,0.000060,0.012304,0.035740
4,4,4,4,4,0.543781,0.996820,1.250040,0.825479,38.552851,0.532117,...,0.028144,0.630038,0.000000,0.050611,0.000135,0.002957,0.621002,0.000078,0.010994,0.159151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028,3028,3028,3028,3028,0.448961,0.989977,1.731302,0.666109,41.934697,0.435118,...,0.026311,0.712928,0.107290,0.045583,0.000015,0.000000,1.839161,0.000000,0.011774,0.034605
3029,3029,3029,3029,3029,0.420328,0.991325,1.813781,0.664752,55.356102,0.405909,...,0.024516,1.122588,0.000000,0.026218,0.000000,0.001101,1.456659,0.000000,0.005547,0.000000
3030,3030,3030,3030,3030,0.393559,0.985864,1.984105,0.646003,66.479941,0.378523,...,0.022063,0.000000,0.511106,0.000000,0.000064,0.010102,1.264072,0.000110,0.005954,0.212550
3031,3031,3031,3031,3031,0.388006,0.992199,1.564622,0.699199,43.493634,0.373067,...,0.020654,3.266987,0.000000,0.020026,0.000000,0.006786,2.593209,0.000083,0.005921,0.043086


In [4]:
split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=1)
for train_index, test_index in split.split(df_1, df_1['0']):
    strat_train_set = df_1.loc[train_index]
    strat_test_set = df_1.loc[test_index]

In [5]:
label_train = strat_train_set['0']
label_test = strat_test_set['0']

strat_train_set.drop(['Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0', 'index','0'],axis=1, inplace=True)
strat_test_set.drop(['Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0', 'index','0'],axis=1, inplace=True)

### VGG GLCM RestNet

In [5]:
#feature selection
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.feature_selection import chi2

fs = SelectKBest(score_func = chi2, k='all')
fs.fit(strat_train_set, label_train)


cols = fs.get_support(indices=True)
list_column_categoric = (strat_train_set.iloc[:,cols].columns)
list_score_select_cat = [fs.scores_[val] for val in range(len(fs.scores_))]

dict_score_cat = {'feature':list_column_categoric, 'score':list_score_select_cat}

score_cat_df = pd.DataFrame(dict_score_cat)


score_cat_df.sort_values(by='score', ascending=False)

,feature,score
14,Contrast3,53677.310063
9,Contrast2,22329.557931
773,40072mobile,5466.025718
619,13111mobile,4131.786834
718,31704mobile,3812.696096
...,...,...
91,340rest,0.000892
152,581rest,0.000654
280,1000rest,0.000523
135,525rest,0.000291


In [6]:
list_ch2 = list(score_cat_df[score_cat_df['score'] > 150]['feature'])
list_feature = list(strat_train_set.columns)
print(len(list_ch2))

252


In [7]:
for i in range(0, len(list_ch2)):
    list_feature.remove(list_ch2[i])

In [8]:
print(f"jumlah feature yang di drop: {len(list_feature)}")

jumlah feature yang di drop: 609


In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class selectionFiture(BaseEstimator, TransformerMixin):
    def __init__(self, drop_columns ):
        self.drop_columns = drop_columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(self.drop_columns, axis=1)

In [10]:
attr_df = selectionFiture(drop_columns=list_feature)
df_drop = attr_df.transform(strat_train_set)

In [11]:
df_drop = attr_df.transform(strat_test_set)

In [19]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator = RandomForestClassifier(random_state=11))

clf.fit(strat_train_set, label_train)

BaggingClassifier(base_estimator=RandomForestClassifier(random_state=11))

In [20]:
some_data_predict = clf.predict(strat_test_set)
print(f'classification report some data RF : \n {classification_report(label_test, some_data_predict)}\naccuracy score : {accuracy_score(label_test, some_data_predict)}')

classification report some data RF : 
               precision    recall  f1-score   support

         0.0       0.85      0.92      0.88       122
         1.0       0.86      0.77      0.81       124
         2.0       0.89      0.92      0.90       119
         3.0       0.96      0.90      0.93       118
         4.0       0.92      0.98      0.95       124

    accuracy                           0.89       607
   macro avg       0.90      0.89      0.89       607
weighted avg       0.90      0.89      0.89       607

accuracy score : 0.8945634266886326


In [21]:
confusion_matrix(label_test, some_data_predict)

array([[112,   9,   0,   1,   0],
       [ 12,  95,  10,   0,   7],
       [  1,   3, 109,   3,   3],
       [  7,   2,   3, 106,   0],
       [  0,   2,   1,   0, 121]])

In [22]:
from sklearn.ensemble import VotingClassifier

clf1 = RandomForestClassifier(random_state=11)
clf2 = BaggingClassifier(base_estimator= RandomForestClassifier(), max_features=1.0, max_samples= 1.0, n_estimators= 50)

eclf = VotingClassifier(estimators=[('rf',clf1), ('bg',clf2)])

eclf = eclf.fit(strat_train_set, label_train)

In [23]:
some_data_predict = eclf.predict(strat_test_set)
print(f'classification report some data RF : \n {classification_report(label_test, some_data_predict)}\naccuracy score : {accuracy_score(label_test, some_data_predict)}')

classification report some data RF : 
               precision    recall  f1-score   support

         0.0       0.85      0.91      0.88       122
         1.0       0.84      0.80      0.82       124
         2.0       0.90      0.93      0.92       119
         3.0       0.98      0.89      0.93       118
         4.0       0.93      0.96      0.94       124

    accuracy                           0.90       607
   macro avg       0.90      0.90      0.90       607
weighted avg       0.90      0.90      0.90       607

accuracy score : 0.8978583196046128


### VGG RESTNET GLCM DENSENEt

In [8]:
#feature selection
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.feature_selection import chi2

fs = SelectKBest(score_func = chi2, k='all')
fs.fit(strat_train_set, label_train)


cols = fs.get_support(indices=True)
list_column_categoric = (strat_train_set.iloc[:,cols].columns)
list_score_select_cat = [fs.scores_[val] for val in range(len(fs.scores_))]

dict_score_cat = {'feature':list_column_categoric, 'score':list_score_select_cat}

score_cat_df = pd.DataFrame(dict_score_cat)


score_cat_df.sort_values(by='score', ascending=False)

,feature,score
14,Contrast3,53677.310063
9,Contrast2,22329.557931
1045,33493dense,5871.305153
773,40072mobile,5466.025718
619,13111mobile,4131.786834
...,...,...
91,340rest,0.000892
152,581rest,0.000654
280,1000rest,0.000523
135,525rest,0.000291


In [24]:
list_ch2 = list(score_cat_df[score_cat_df['score'] > 55]['feature'])
list_feature = list(strat_train_set.columns)
print(len(list_ch2))

487


In [25]:
for i in range(0, len(list_ch2)):
    list_feature.remove(list_ch2[i])

In [26]:
print(f"jumlah feature yang di drop: {len(list_feature)}")

jumlah feature yang di drop: 646


In [27]:
from sklearn.base import BaseEstimator, TransformerMixin

class selectionFiture(BaseEstimator, TransformerMixin):
    def __init__(self, drop_columns ):
        self.drop_columns = drop_columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(self.drop_columns, axis=1)

In [28]:
attr_df = selectionFiture(drop_columns=list_feature)
df_drop = attr_df.transform(strat_train_set)

In [29]:
df_drop = attr_df.transform(strat_test_set)

#### Bagging

In [32]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator = RandomForestClassifier(random_state=11))
clf.fit(strat_train_set, label_train)

BaggingClassifier(base_estimator=RandomForestClassifier(random_state=11))

In [53]:
some_data_predict = clf.predict(strat_test_set)
print(f'''{classification_report(label_test, some_data_predict)}
      \naccuracy score : {accuracy_score(label_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(label_test, some_data_predict)}''')

              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90       122
         1.0       0.85      0.81      0.83       124
         2.0       0.90      0.94      0.92       119
         3.0       0.97      0.90      0.93       118
         4.0       0.95      0.98      0.96       124

    accuracy                           0.91       607
   macro avg       0.91      0.91      0.91       607
weighted avg       0.91      0.91      0.91       607

      
accuracy score : 0.9077429983525536
      
cnfusion matrix : 
[[112   9   0   1   0]
 [  9 100   9   0   6]
 [  1   4 112   1   1]
 [  6   2   4 106   0]
 [  0   2   0   1 121]]


all :  0.9060955518945635
60 : 0.9028006589785832
50 : 0.9028006589785832
55 : 0.9044481054365733

#### Votting

In [45]:
#VOTING
from sklearn.ensemble import VotingClassifier

clf1 = RandomForestClassifier()
clf2 = BaggingClassifier(base_estimator= RandomForestClassifier(), max_features=1.0, max_samples= 1.0, n_estimators= 50)

eclf = VotingClassifier(estimators=[('rf',clf1), ('bg',clf2)])

eclf = eclf.fit(strat_train_set, label_train)

In [52]:
some_data_predict = eclf.predict(strat_test_set)
print(f'{classification_report(label_test, some_data_predict)}\naccuracy score : {accuracy_score(label_test, some_data_predict)}')

              precision    recall  f1-score   support

         0.0       0.85      0.93      0.89       122
         1.0       0.86      0.82      0.84       124
         2.0       0.90      0.94      0.92       119
         3.0       0.98      0.86      0.92       118
         4.0       0.95      0.96      0.96       124

    accuracy                           0.90       607
   macro avg       0.91      0.90      0.90       607
weighted avg       0.91      0.90      0.90       607

accuracy score : 0.9044481054365733


all : 0.9044481054365733

#### RandomForest

In [47]:
#RANDOMFOREST
rf = RandomForestClassifier()
rf.fit(strat_train_set, label_train)

RandomForestClassifier()

In [50]:
some_data_predict = rf.predict(strat_test_set)
print(f'classification report some data votting : \n {classification_report(label_test, some_data_predict)}\naccuracy score : {accuracy_score(label_test, some_data_predict)}')

classification report some data votting : 
               precision    recall  f1-score   support

         0.0       0.87      0.93      0.90       122
         1.0       0.87      0.84      0.85       124
         2.0       0.91      0.94      0.93       119
         3.0       0.97      0.88      0.92       118
         4.0       0.94      0.96      0.95       124

    accuracy                           0.91       607
   macro avg       0.91      0.91      0.91       607
weighted avg       0.91      0.91      0.91       607

accuracy score : 0.9093904448105437


all : 0.9093904448105437

#### AdaBoost

In [49]:
#ADOBOOSTCLASIFIER
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(strat_train_set, label_train)

AdaBoostClassifier()

In [51]:
some_data_predict = ada.predict(strat_test_set)
print(f'{classification_report(label_test, some_data_predict)}\naccuracy score : {accuracy_score(label_test, some_data_predict)}')

              precision    recall  f1-score   support

         0.0       0.87      0.85      0.86       122
         1.0       0.60      0.69      0.64       124
         2.0       0.66      0.67      0.66       119
         3.0       0.79      0.74      0.76       118
         4.0       0.96      0.88      0.92       124

    accuracy                           0.77       607
   macro avg       0.78      0.77      0.77       607
weighted avg       0.78      0.77      0.77       607

accuracy score : 0.7660626029654036


all : 0.7660626029654036

### +Efficient

#### All

In [4]:
df = pd.read_csv('ready.csv')
df

,Unnamed: 0.6,Unnamed: 0,211dense,296dense,402dense,432dense,466dense,892dense,1188dense,1202dense,...,31929vgg,32006vgg,32100vgg,32241vgg,32268vgg,32539vgg,32548vgg,32689vgg,32694vgg,label
0,0,0,0.002113,0.002889,0.002463,0.014652,0.000344,0.000000,0.200305,0.000166,...,0.236666,0.000000,0.000000,0.0,0.214589,0.000000,0.064896,0.061991,0.305349,0
1,1,1,0.001438,0.002601,0.002602,0.014899,0.000337,0.000000,0.103333,0.000259,...,0.179535,0.000000,0.000000,0.0,0.219380,0.000000,0.060197,0.045982,0.308021,0
2,2,2,0.003572,0.002963,0.002506,0.015130,0.000379,0.000000,0.175900,0.000244,...,0.157752,0.047314,0.000000,0.0,0.266565,0.004380,0.032139,0.044623,0.245007,0
3,3,3,0.001439,0.002818,0.002481,0.014944,0.000304,0.000000,0.173830,0.000115,...,0.108208,0.000000,0.000000,0.0,0.221949,0.000000,0.068435,0.079967,0.285927,0
4,4,4,0.000434,0.002497,0.002678,0.014119,0.000335,0.000000,0.158985,0.000438,...,0.057059,0.000000,0.000000,0.0,0.263825,0.000000,0.069721,0.072631,0.249920,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4042,4042,4042,0.000000,0.000000,0.001332,0.008768,0.000035,0.000000,0.000000,0.000395,...,0.330900,0.690772,0.399666,0.0,0.329033,0.061612,0.000000,0.000000,0.086072,4
4043,4043,4043,0.001417,0.002338,0.002575,0.013246,0.000130,0.059455,0.016325,0.000177,...,0.918796,0.547085,0.000000,0.0,0.461605,0.125676,0.000000,0.000000,0.267543,4
4044,4044,4044,0.001356,0.002733,0.002593,0.014656,0.000320,0.000000,0.164176,0.000201,...,0.000000,1.027705,0.257086,0.0,0.361025,0.070178,0.000000,0.073477,0.331383,4
4045,4045,4045,0.001143,0.003064,0.002579,0.011211,0.000000,0.000000,0.050476,0.000306,...,0.186353,0.000000,0.000000,0.0,0.218310,0.000000,0.064177,0.058530,0.305462,4


In [5]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3'
      ,'Unnamed: 0.4', 'Unnamed: 0.5', 'Unnamed: 0.6'], axis=1, inplace=True)

In [6]:
y = df['label'].copy()
X = df.drop(['label'], axis=1).copy()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.05, 
                                                    random_state=46)

In [7]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator = RandomForestClassifier(random_state=11))
clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(random_state=11))

In [10]:
some_data_predict = clf.predict(X_test)
print(f'''{classification_report(y_test, some_data_predict)}
      \naccuracy score : {accuracy_score(y_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(y_test, some_data_predict)}''')

              precision    recall  f1-score   support

           0       0.83      0.93      0.88        43
           1       0.88      0.81      0.84        43
           2       0.95      0.91      0.93        46
           3       0.93      0.96      0.95        28
           4       1.00      0.98      0.99        43

    accuracy                           0.92       203
   macro avg       0.92      0.92      0.92       203
weighted avg       0.92      0.92      0.92       203

      
accuracy score : 0.916256157635468
      
cnfusion matrix : 
[[40  3  0  0  0]
 [ 6 35  2  0  0]
 [ 1  1 42  2  0]
 [ 1  0  0 27  0]
 [ 0  1  0  0 42]]


In [9]:

#VOTING
from sklearn.ensemble import VotingClassifier, BaggingClassifier

clf1 = RandomForestClassifier()
clf2 = BaggingClassifier(base_estimator= RandomForestClassifier(), max_features=1.0, max_samples= 1.0, n_estimators= 50)

eclf = VotingClassifier(estimators=[('rf',clf1), ('bg',clf2)])

eclf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('bg',
                              BaggingClassifier(base_estimator=RandomForestClassifier(),
                                                n_estimators=50))])

In [11]:
some_data_predict1 = eclf.predict(X_test)
print(f'''{classification_report(y_test, some_data_predict1)}
      \naccuracy score : {accuracy_score(y_test, some_data_predict1)}
      \ncnfusion matrix : \n{confusion_matrix(y_test, some_data_predict1)}''')

              precision    recall  f1-score   support

           0       0.82      0.98      0.89        43
           1       0.92      0.81      0.86        43
           2       0.98      0.91      0.94        46
           3       0.93      0.96      0.95        28
           4       1.00      0.98      0.99        43

    accuracy                           0.93       203
   macro avg       0.93      0.93      0.93       203
weighted avg       0.93      0.93      0.93       203

      
accuracy score : 0.9261083743842364
      
cnfusion matrix : 
[[42  1  0  0  0]
 [ 7 35  1  0  0]
 [ 1  1 42  2  0]
 [ 1  0  0 27  0]
 [ 0  1  0  0 42]]


In [25]:
#RANDOMFOREST
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [26]:
some_data_predict = rf.predict(X_test)
print(f'''{classification_report(y_test, some_data_predict)}
      \naccuracy score : {accuracy_score(y_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(y_test, some_data_predict)}''')

              precision    recall  f1-score   support

           0       0.86      0.98      0.91        43
           1       0.97      0.84      0.90        43
           2       0.93      0.93      0.93        46
           3       0.93      0.93      0.93        28
           4       1.00      1.00      1.00        43

    accuracy                           0.94       203
   macro avg       0.94      0.94      0.94       203
weighted avg       0.94      0.94      0.94       203

      
accuracy score : 0.9359605911330049
      
cnfusion matrix : 
[[42  1  0  0  0]
 [ 5 36  2  0  0]
 [ 1  0 43  2  0]
 [ 1  0  1 26  0]
 [ 0  0  0  0 43]]


In [27]:
#ADOBOOSTCLASIFIER
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)

AdaBoostClassifier()

In [28]:
some_data_predict = ada.predict(X_test)
print(f'''{classification_report(y_test, some_data_predict)}
      \naccuracy score : {accuracy_score(y_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(y_test, some_data_predict)}''')

              precision    recall  f1-score   support

           0       0.80      0.95      0.87        43
           1       0.71      0.58      0.64        43
           2       0.68      0.89      0.77        46
           3       0.83      0.54      0.65        28
           4       1.00      0.91      0.95        43

    accuracy                           0.79       203
   macro avg       0.81      0.77      0.78       203
weighted avg       0.80      0.79      0.79       203

      
accuracy score : 0.7931034482758621
      
cnfusion matrix : 
[[41  2  0  0  0]
 [ 8 25  8  2  0]
 [ 1  3 41  1  0]
 [ 1  2 10 15  0]
 [ 0  3  1  0 39]]


#### select

In [7]:
df_1= pd.read_csv('full_train_dt_efficient.csv')
label_1 = df_1['0']

In [8]:
df_1

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,Energy,Corr,Diss_sim,Homogen,Contrast,...,120743efficient,120879efficient,122406efficient,122650efficient,122691efficient,122918efficient,124134efficient,124250efficient,124555efficient,125207efficient
0,0,0,0,0,0,0.569125,0.995196,1.267977,0.787601,29.518778,...,-0.251578,1.148612,-0.245898,-0.122847,-0.242252,5.292639,5.135893,5.462677,5.282661,4.751120
1,1,1,1,1,1,0.527938,0.995842,1.471252,0.822016,57.029869,...,-0.251559,1.142557,-0.245945,-0.122596,-0.242328,5.299435,5.131536,5.466395,5.285784,4.744389
2,2,2,2,2,2,0.545918,0.996925,1.381366,0.830510,46.555213,...,-0.251375,1.148002,-0.245844,-0.122479,-0.242329,5.294795,5.133411,5.461254,5.279757,4.747968
3,3,3,3,3,3,0.596634,0.996485,1.356422,0.838555,50.394299,...,-0.251470,1.142004,-0.245529,-0.122615,-0.242318,5.295320,5.140662,5.467499,5.284116,4.754744
4,4,4,4,4,4,0.543781,0.996820,1.250040,0.825479,38.552851,...,-0.251400,1.148902,-0.245747,-0.122391,-0.242343,5.296155,5.134854,5.462451,5.281322,4.748119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028,3028,3028,3028,3028,3028,0.448961,0.989977,1.731302,0.666109,41.934697,...,-0.251734,1.149225,-0.245690,-0.123225,-0.242964,5.291068,5.134604,5.459740,5.282078,4.750716
3029,3029,3029,3029,3029,3029,0.420328,0.991325,1.813781,0.664752,55.356102,...,-0.251732,1.152119,-0.245883,-0.122897,-0.243012,5.288969,5.129421,5.457845,5.282534,4.747731
3030,3030,3030,3030,3030,3030,0.393559,0.985864,1.984105,0.646003,66.479941,...,-0.251509,1.149539,-0.245508,-0.122995,-0.243214,5.290375,5.131971,5.456860,5.279867,4.754733
3031,3031,3031,3031,3031,3031,0.388006,0.992199,1.564622,0.699199,43.493634,...,-0.251492,1.150785,-0.245963,-0.122825,-0.242654,5.291928,5.129191,5.456686,5.279892,4.745414


In [4]:
df_1.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0', 'index', '0'],axis=1, inplace=True)

In [7]:
#feature selection
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.feature_selection import chi2

fs = SelectKBest(score_func = mutual_info_classif, k='all')
fs.fit(df_1, label_1)


cols = fs.get_support(indices=True)
list_column_categoric = (df_1.iloc[:,cols].columns)
list_score_select_cat = [fs.scores_[val] for val in range(len(fs.scores_))]

dict_score_cat = {'feature':list_column_categoric, 'score':list_score_select_cat}

score_cat_df = pd.DataFrame(dict_score_cat)


score_cat_df.sort_values(by='score', ascending=False)

,feature,score
1342,99708efficient,0.483039
3,Homogen,0.482617
1349,101896efficient,0.458185
1269,66595efficient,0.447552
1308,87512efficient,0.443093
...,...,...
515,25680vgg,0.000000
755,39073mobile,0.000000
605,9495mobile,0.000000
503,24410vgg,0.000000


In [8]:
drop = score_cat_df[score_cat_df['score'] != 0]
drop.head()
print(f"jumlah feature : {len(score_cat_df[score_cat_df['score'] != 0])}")

jumlah feature : 1370


In [9]:
list_df = list(df_1.columns)
drop_list = list(drop.feature)

for i in range(0, len(drop_list)):
    list_df.remove(drop_list[i])

In [10]:
print(f"jumlah feature yang di drop: {len(list_df)}")

jumlah feature yang di drop: 32


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class selectionFiture(BaseEstimator, TransformerMixin):
    def __init__(self, drop_columns ):
        self.drop_columns = drop_columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(self.drop_columns, axis=1)

In [12]:
attr_df = selectionFiture(drop_columns=list_df)
df_1_drop = attr_df.transform(df_1)

In [13]:
df_1_drop.columns

Index(['Energy', 'Corr', 'Diss_sim', 'Homogen', 'Contrast', 'Energy2', 'Corr2',
       'Diss_sim2', 'Homogen2', 'Contrast2',
       ...
       '120743efficient', '120879efficient', '122406efficient',
       '122650efficient', '122691efficient', '122918efficient',
       '124134efficient', '124250efficient', '124555efficient',
       '125207efficient'],
      dtype='object', length=1370)

In [14]:
df_1= pd.read_csv('full_train_dt_efficient.csv')

In [15]:
split = StratifiedShuffleSplit(n_splits = 1, test_size=0.1, random_state=1)
for train_index, test_index in split.split(df_1, df_1['0']):
    strat_train_set = df_1.loc[train_index]
    strat_test_set = df_1.loc[test_index]

In [16]:
label_train = strat_train_set['0']
label_test = strat_test_set['0']

strat_train_set.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0', 'index','0'],axis=1, inplace=True)
strat_test_set.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0', 'index','0'],axis=1, inplace=True)

In [17]:
attr_df = selectionFiture(drop_columns=list_df)
strat_train_set = attr_df.transform(strat_train_set)

In [18]:
attr_df = selectionFiture(drop_columns=list_df)
strat_test_set = attr_df.transform(strat_test_set)

In [19]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator = RandomForestClassifier(random_state=10))
clf.fit(strat_train_set, label_train)

BaggingClassifier(base_estimator=RandomForestClassifier(random_state=10))

In [20]:
some_data_predict = clf.predict(strat_test_set)
print(f'''{classification_report(label_test, some_data_predict)}
      \naccuracy score : {accuracy_score(label_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(label_test, some_data_predict)}''')

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.89        61
         1.0       0.86      0.81      0.83        62
         2.0       0.90      0.95      0.93        60
         3.0       0.96      0.92      0.94        59
         4.0       0.97      1.00      0.98        62

    accuracy                           0.91       304
   macro avg       0.91      0.91      0.91       304
weighted avg       0.91      0.91      0.91       304

      
accuracy score : 0.9144736842105263
      
cnfusion matrix : 
[[55  5  0  1  0]
 [ 6 50  5  0  1]
 [ 0  1 57  1  1]
 [ 2  2  1 54  0]
 [ 0  0  0  0 62]]


In [21]:
#VOTING
from sklearn.ensemble import VotingClassifier

clf1 = RandomForestClassifier(random_state=8)
clf2 = BaggingClassifier(base_estimator= RandomForestClassifier(), max_features=1.0, max_samples= 1.0, n_estimators= 50)

eclf = VotingClassifier(estimators=[('rf',clf1), ('bg',clf2)])

eclf = eclf.fit(strat_train_set, label_train)

In [22]:
some_data_predict = eclf.predict(strat_test_set)
print(f'''{classification_report(label_test, some_data_predict)}
      \naccuracy score : {accuracy_score(label_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(label_test, some_data_predict)}''')

              precision    recall  f1-score   support

         0.0       0.88      0.93      0.90        61
         1.0       0.88      0.82      0.85        62
         2.0       0.89      0.95      0.92        60
         3.0       0.98      0.88      0.93        59
         4.0       0.97      1.00      0.98        62

    accuracy                           0.92       304
   macro avg       0.92      0.92      0.92       304
weighted avg       0.92      0.92      0.92       304

      
accuracy score : 0.9177631578947368
      
cnfusion matrix : 
[[57  3  0  1  0]
 [ 6 51  4  0  1]
 [ 0  2 57  0  1]
 [ 2  2  3 52  0]
 [ 0  0  0  0 62]]


In [23]:
#RANDOMFOREST
rf = RandomForestClassifier(random_state=8)
rf.fit(strat_train_set, label_train)

RandomForestClassifier(random_state=8)

In [24]:
some_data_predict = rf.predict(strat_test_set)
print(f'''{classification_report(label_test, some_data_predict)}
      \naccuracy score : {accuracy_score(label_test, some_data_predict)}
      \ncnfusion matrix : \n{confusion_matrix(label_test, some_data_predict)}''')

              precision    recall  f1-score   support

         0.0       0.95      0.90      0.92        61
         1.0       0.84      0.85      0.85        62
         2.0       0.88      0.93      0.90        60
         3.0       0.96      0.88      0.92        59
         4.0       0.95      1.00      0.98        62

    accuracy                           0.91       304
   macro avg       0.92      0.91      0.91       304
weighted avg       0.92      0.91      0.91       304

      
accuracy score : 0.9144736842105263
      
cnfusion matrix : 
[[55  5  0  1  0]
 [ 2 53  5  0  2]
 [ 0  2 56  1  1]
 [ 1  3  3 52  0]
 [ 0  0  0  0 62]]
